In [ ]:

!pip install nltk

In [ ]:

from google.colab import files
uploaded = files.upload()

Saving Fake.csv to Fake.csv
Saving True.csv to True.csv


In [ ]:

import pandas as pd

# Read CSVs
fake_df = pd.read_csv("Fake.csv")
real_df = pd.read_csv("True.csv")

# Add labels
fake_df["label"] = 0  # Fake
real_df["label"] = 1  # Real

# Combine
data = pd.concat([fake_df, real_df], axis=0)
data = data.sample(frac=1).reset_index(drop=True)

# Keep only text and label
data = data[["text", "label"]]
data.head()

,text,label
0,If Texas Lt. Governor Dan Patrick (R) thought ...,0
1,BERLIN (Reuters) - Germany s Social Democrat (...,1
2,(Reuters) - U.S. President Donald Trump said o...,1
3,21st Century Wire says How seriously can we ta...,0
4,When did white students become throw away citi...,0


In [ ]:

import re
import string
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:

def clean_text(text):
    text = text.lower()  # lowercase
    text = re.sub('\[.*?\]', '', text)  # remove [text]
    text = re.sub('\\W', ' ', text)  # remove non-word chars
    text = re.sub('https?://\S+|www\.\S+', '', text)  # remove URLs
    text = re.sub('<.*?>+', '', text)  # remove HTML tags
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)  # remove punctuation
    text = re.sub('\n', '', text)  # remove newlines
    text = re.sub('\w*\d\w*', '', text)  # remove digits
    return text

In [ ]:

data["text"] = data["text"].apply(clean_text)
data.head()

,text,label
0,if texas lt governor dan patrick r thought ...,0
1,berlin reuters germany s social democrat ...,1
2,reuters u s president donald trump said o...,1
3,century wire says how seriously can we take a...,0
4,when did white students become throw away citi...,0


In [ ]:

from sklearn.model_selection import train_test_split

X = data["text"]
y = data["label"]

# 80% training, 20% testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:

from sklearn.feature_extraction.text import TfidfVectorizer

# Create TF-IDF object
tfidf = TfidfVectorizer(stop_words='english', max_df=0.85, ngram_range=(1, 2))


# Fit and transform
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

In [ ]:

from sklearn.linear_model import LogisticRegression

model = LogisticRegression(max_iter=1000)
model.fit(X_train_tfidf, y_train)

LogisticRegression(max_iter=1000)

In [ ]:

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Predict
y_pred = model.predict(X_test_tfidf)

# Accuracy
print("Accuracy:", accuracy_score(y_test, y_pred))

# Confusion matrix
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

# Full report
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.9866369710467706
Confusion Matrix:
 [[4607   85]
 [  35 4253]]

Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.98      0.99      4692
           1       0.98      0.99      0.99      4288

    accuracy                           0.99      8980
   macro avg       0.99      0.99      0.99      8980
weighted avg       0.99      0.99      0.99      8980



In [17]:
import joblib

# Save model normally
with open('model.pkl', 'wb') as f:
    pickle.dump(model, f)

# Compress tfidf.pkl
joblib.dump(tfidf, 'tfidf_compressed.pkl', compress=3)

# Download
from google.colab import files
files.download('tfidf_compressed.pkl')
files.download('model.pkl')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>